In [1]:
from random import choices, randint, randrange, random
from typing import List, Tuple

In [2]:
Population = List[int]
population_length = 4

## Conversões

In [83]:
def int_to_bit(valor: int) -> List[int]:
  res = []
  for i in range(0,32):
    if valor & (0x01<<i) > 0:
      res.insert(0,1)
    else:
      res.insert(0,0)
  return res

In [84]:
def bit_to_int(bit: List[int]):
  num = 0
  if bit[0] == 1:
    # num negativos
    for i in range(1,32):
      if bit[i]==0:
        num = num + 2**(31-i)
    num = -1 - num
  else:
    for i in range(1,32):
      if bit[i]>0:
        num = num + 2**(31-i)
  return num

## Inicialização da População

In [85]:
def generate_population(population_length: int) -> Population:
  return [randint(-10, 10) for _ in range(population_length)]

## Avaliação de cada individuo

In [86]:
def population_fitness(population: Population) -> List[int]:
  fitness = []
  for value in population:
    fit = value * value -3 * value + 4
    fitness.append(abs(fit))
  return fitness

## Seleção de alguns individuos


In [87]:
def selection_pair(population: Population, fitness: List[int]) -> Tuple[int, int]:
  # Irá selecionar os 2 melhores genomas
  zipped_fitness = list(zip([_ for _ in range(len(fitness))],fitness))
  zipped_fitness = sorted(zipped_fitness, key = lambda x: x[1])

  selected_index = []
  for index in zipped_fitness[:2]:
    selected_index.append(index[0])

  return [population[selected_index[0]], population[selected_index[1]]], fitness[selected_index[0]]

## Nova gen

In [88]:
def new_generation(best_pop: List[int], pop_size: int = 4) -> Population:
  # Criando a nova população
  new_population = best_pop.copy()

  # # Loop para gerar os demais genomas além dos 2 best fit
  while(len(new_population) < pop_size):
    new_gen_a, new_gen_b = crossover(best_pop)
    new_population.append(new_gen_a)
    new_population.append(new_gen_b)

  #   # Validações para gerar a nova população do mesmo tamanho que a original
    if len(new_population) > pop_size:
      new_population.pop()
  return new_population

## Cross-over e mutação

In [89]:
def crossover(pair: List[int], cross_rate: float = 0.6) -> int:
  if len(pair) != 2:
    raise Exception(f"pair diferente de 2: {len(pair)}")

  if random() >= cross_rate:
    bit_a =  int_to_bit(pair[0])
    bit_b =  int_to_bit(pair[1])    
    novo_a = None
    novo_b = None
    valores_validos = 0
    while valores_validos < 2:
      corte = randint(1,30)
      novo_bit_a = bit_a[0:corte]
      novo_bit_a.extend(bit_b[corte:])
      novo_bit_b = bit_b[0:corte]
      novo_bit_b.extend(bit_a[corte:])

      novo_int_a = bit_to_int(novo_bit_a)
      novo_int_b = bit_to_int(novo_bit_b)
      
      if novo_a is None:
        if novo_int_a >= -10 and novo_int_a <= 10:
          valores_validos = valores_validos + 1
          novo_a = novo_int_a

      if novo_b is None:
        if novo_int_b >= -10 and novo_int_b <= 10:
          valores_validos = valores_validos + 1
          novo_b = novo_int_b
    return [novo_a, novo_b]
  else:
    return pair

In [90]:
def mutation(value: int, probability: float = 0.99) -> int:
  if random() >= probability:
    valido = False
    while True:
      new_value = randint(-10, 10)
      if new_value != value:
        return new_value
  else:
    return value

## Execução

In [91]:
def run_gen(population: Population) -> Population:
  fitness = population_fitness(population)
  best_pair, best_fit = selection_pair(population, fitness)
  new_pop = new_generation(best_pair)
  mutated_pop = []
  for value in new_pop:
    mutated_pop.append(mutation(value))  

  return best_pair[0], best_fit, mutated_pop

In [96]:
population = generate_population(population_length)
top_value = None
top_fit = None
on_geneneration = None

In [98]:
for gen in range(5):
  best_genome, best_fit, population = run_gen(population)
  print(f'Generation: {gen} | Best Fit: {best_fit} | Valor: {best_genome}')
  if top_fit is None:
    top_fit = best_fit
    top_genome = best_genome
    on_geneneration = gen
  elif best_fit < top_fit:
    top_fit = best_fit
    top_genome = best_genome
    on_geneneration = gen

  if best_fit == 0:
    break

print(f'Best results found on {on_geneneration} generation with ' \
  f'fitness of {top_fit} on {top_genome} value')

Generation: 0 | Best Fit: 4 | Valor: 0
Generation: 1 | Best Fit: 4 | Valor: 0
Generation: 2 | Best Fit: 2 | Valor: 2
Generation: 3 | Best Fit: 2 | Valor: 2
Generation: 4 | Best Fit: 2 | Valor: 2
Best results found on 2 generation with fitness of 2 on 2 value
